In [ ]:
from json import loads

import numpy as np
import xarray as xr
import zarr
from dask.distributed import Client
from fsspec.implementations.http import HTTPFileSystem
from xarray.testing import assert_chunks_equal, assert_equal, assert_identical

import xpublish

In [ ]:
# Let's check to make sure our server started alright
!head ../logfile.txt

In [ ]:
# Start a dask cluster for use on the client side
client = Client(n_workers=4, dashboard_address=43757)
client

You can now open three more browser tabs/windows:

_Note that you will have to modify the url prefix slightly, to do this, just copy the first part of your browser's URL_

1. Xpublish Web App: e.g. https://hub.gke.mybinder.org/user/jhamman-xpublish-gbbqbxfi/proxy/9000
2. Xpublish's Dask Cluster Dashboard:  e.g. https://hub.gke.mybinder.org/user/jhamman-xpublish-gbbqbxfi/proxy/8787/status
3. This Notebook's Dask Cluster Dashboard: e.g. https://hub.gke.mybinder.org/user/jhamman-xpublish-gbbqbxfi/proxy/43757/status

_Also note that this port numbers may change. The server side ports are available in `logfile.txt` (see above) and the client-side port is in the cell above._

In [ ]:
# We can access our API using fsspec's HTTPFileSystem
fs = HTTPFileSystem()

# The http mapper gives us a dict-like interface to the API
http_map = fs.get_mapper("http://0.0.0.0:9000")

In [ ]:
# We can access API enpoints by key now...
for key in [".zmetadata", "keys"]:
    print(key, http_map[key], "\n")

In [ ]:
# The .zmetadata key returns the json dictionary of consolidated zarr metadata
# We can load/decode that and access one array's attributes
d = loads(http_map[".zmetadata"])
d["metadata"]["air/.zattrs"]

In [ ]:
# We can pass that mapper object directly to zarr's open_consolidated function
# This returns a zarr groups
zg = zarr.open_consolidated(http_map, mode="r")
zg.tree()

In [ ]:
# And we can do the same with xarray's open_zarr function
ds = xr.open_zarr(http_map, consolidated=True)
ds

In [ ]:
# The rest of this notebook applies some simple tests to show that the served dataset is indentical to the
# "air_temperature" dataset in xarray's tutorial dataset.
ds_tutorial = xr.tutorial.open_dataset(
    "air_temperature", chunks=dict(lat=5, lon=5), decode_cf=True
)

In [ ]:
ds_tutorial.air.attrs

In [ ]:
def test(actual, expected, index):
    """a simple equality test with index as a parameter"""
    assert np.array_equal(actual[index].values, expected[index].values)

In [ ]:
# test a bunch of indexing patterns
for index in [
    (0, 0, 0),
    (slice(0, 4), 0, 0),
    (slice(0, 4), slice(0, 4), 0),
    (slice(0, 4), slice(0, 4), slice(0, 4)),
    (slice(-4), slice(0, 4), slice(0, 4)),
    (slice(None), slice(0, 4), slice(0, 4)),
    (slice(None), slice(None), slice(0, 4)),
    (slice(None), slice(None), slice(None)),
]:
    print(index)
    test(ds_tutorial["air"], ds["air"], index)

In [ ]:
assert_equal(ds, ds_tutorial)
assert_chunks_equal(ds, ds_tutorial)
assert_identical(ds, ds_tutorial)